In [ ]:
#!/usr/bin/env python3
"""
Evaluation script for pre-trained reinforcement learning models.

This script provides functionality for loading trained RL models, generating trajectories,
analyzing rewards, and creating visualizations for research papers/theses.
"""

# Standard library imports
import argparse
import copy
import json
import logging
import os
import pickle
import sys
import time
import warnings
from datetime import datetime
from itertools import combinations
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union

# Third-party imports
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import yaml
from IPython.display import display
from scipy.spatial.distance import pdist
from scipy.stats import entropy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Project-specific imports
sys.path.append('../.')

# Import support modules as namespaces (avoid wildcard imports)
import BayesOpt_Support as bayesopt_support
import REINFORCE_Support as reinforce_support
import SMCMC_Support as smcmc_support
import testing_utils as testing_utils

# Environment and utility imports
from environments import GeneralEnvironment
from mlflow_logger import MLflowLogger
from utility_functions import (
    calculate_cosine_diversity, 
    get_policy_dist, 
    load_initial_state,
    load_entire_model, 
    plot_distributions_per_feature,
    setup_logger, 
    simulate_trajectories,
    seed_all
)


# Setup

In [ ]:

def analyze_and_visualize_results(
    agent_name: str,
    trained_agent,
    training_rewards: list,
    env,
    feature_names: list,
    training_parameters: dict,
    output_path: str,
    model_identifier: str
):
    """
    Analyze and visualize the results of a trained RL agent.
    
    Args:
        agent_name (str): Name of the algorithm (e.g., 'SAC', 'REINFORCE')
        trained_agent: The trained agent object
        training_rewards (list): List of rewards from training
        env: Training environment
        feature_names (list): Names of the features
        training_parameters (dict): Training parameters
        output_path (str): Path to save outputs
        model_identifier (str): Unique identifier for the model
    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    # Plot training progress
    plt.figure(figsize=(10, 6))
    plt.plot(training_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.title(f"{agent_name} Training Progress")
    plt.savefig(f"{output_path}/{agent_name.lower()}_training_progress_{model_identifier}.png")
    plt.close()

    # Generate high-reward trajectories
    high_reward_trajectories = generate_high_reward_trajectories(
        env,
        trained_agent=trained_agent,
        num_trajectories=200,
        max_steps=training_parameters["trajectory_length"],
    )

    # Generate top 10 summary
    top10_df, _, _ = create_top10_summary_table(
        high_reward_trajectories, ['Timestamp'] + feature_names
    )

    # Save high reward trajectories
    with open(f"{output_path}/{agent_name.lower()}_high_reward_trajectories_{model_identifier}.pkl", 'wb') as f:
        pickle.dump(high_reward_trajectories, f)
    # Save styled top10 table
    top10_html_path = f"{output_path}/{agent_name.lower()}_top10_cases_{model_identifier}.html"
    top10_df.style.background_gradient(cmap='Blues').to_html(top10_html_path)
    display(top10_df.style.background_gradient(cmap='Blues'))

    # Keep top 50 trajectories for diversity analysis
    # MAke sure final reward is numeric
    
    # Calculate diversity metrics
    top10_df['FinalReward'] = top10_df['FinalReward'].apply(lambda x: x[0])
    top50_df = top10_df.nlargest(50, 'FinalReward')
    final_states = top50_df[feature_names].values
    avg_div, avg_div_normalized, distances = calculate_euclidean_diversity(final_states, top50_df['FinalReward'].values)
    print(f"Average Euclidean Diversity: {avg_div}")
    print(f"Normalized Euclidean Diversity: {avg_div_normalized}")

    # Plot rewards distribution
    last_step_rewards = top10_df['FinalReward'].values
    pd.Series(last_step_rewards, name='Final Rewards').to_csv(f"{output_path}/{agent_name}_Final_Rewards.csv", index=False)
    
    plt.figure(figsize=(12, 6))
    sns.histplot(data=last_step_rewards, bins=50, kde=True, stat='density')
    
    plt.axvline(np.mean(last_step_rewards), color='red', linestyle='dashed', linewidth=2, 
                label=f'Mean: {np.mean(last_step_rewards):.2f}')
    plt.axvline(np.median(last_step_rewards), color='green', linestyle='dashed', linewidth=2, 
                label=f'Median: {np.median(last_step_rewards):.2f}')

    stats_text = (f'Statistics:\n'
                 f'Mean: {np.mean(last_step_rewards):.2f}\n'
                 f'Median: {np.median(last_step_rewards):.2f}\n'
                 f'Std: {np.std(last_step_rewards):.2f}\n'
                 f'Max: {np.max(last_step_rewards):.2f}\n'
                 f'Min: {np.min(last_step_rewards):.2f}')

    plt.text(0.9, 0.75, stats_text, transform=plt.gca().transAxes,
             verticalalignment='center', horizontalalignment='left',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.xlabel('Final Reward Values', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.title(f'Distribution of Final Rewards ({agent_name})', fontsize=14, pad=20)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{output_path}/{agent_name.lower()}_rewards_distribution_{model_identifier}.png")
    plt.close()

    # Plot diversity heatmap
    euclidean_distances = squareform(distances)
    plt.figure(figsize=(8, 8))
    sns.heatmap(euclidean_distances, 
                cmap='viridis',
                fmt=".2f",
                vmin=0,
                vmax=500,
                cbar_kws={'label': 'Normalized Euclidean Distance'})

    plt.title(f"Euclidean Distance Matrix for Top 10 Trajectories\n"
              f"Diversity Score: {avg_div:.4f}, Normalized: {avg_div_normalized:.4f}")
    plt.xlabel("Trajectory Index")
    plt.ylabel("Trajectory Index")
    plt.tight_layout()
    # Save as high quality pdf 
    plt.savefig(f"{output_path}/{agent_name.lower()}_diversity_heatmap_{model_identifier}.pdf", dpi=300)
    plt.close()

# Usage example:

In [ ]:
# Define the environment
from utility_functions import  seed_all
import random
CONFIG_PATH = r'config/run_params.yaml'
# MODEL_PATH = "oracles/oracle_3.joblib"
MODEL_PATH = "oracles/oracle_3_GradientBoostingRegressor.joblib"
sys.path.append('../.')
config, env, logger, training_parameters, initial_state, feature_names = setup_environment_and_logger(CONFIG_PATH, MODEL_PATH)
# Correct state_dim to exclude the time-step part for action selection
state_dim = len(initial_state) + 1   # Include time-step part for the environment
action_dim = len(initial_state)     # Exclude time-step part for actions
output_path = r"results/PaperPlots_O3_REINFORCE_GBR"
results_runs = {}
for run in range(30):
    print(f"Run {run+1}/30")
    # Simulate trajectories using the forward and backward models
    random_seed = random.randint(0, 10000)    
    # with Temperature
    trained_agent,reinforce_rewards= train_rl_agent(
        env=env,
        state_dim=state_dim,
        action_dim=action_dim,
        algorithm='reinforce',
        training_episodes=500,
        max_steps=training_parameters["trajectory_length"],
        sac_params = {
            "alpha": 0.5,  # Temperature parameter for SAC
            "tau": 0.0005,  # Soft update parameter for SAC
            "gamma": 0.999,  # Discount factor for SAC
        },
        seed =  random_seed #33#34
    )
    # Store the trained agent and rewards
    results_runs[run] = {
        'trained_agent': trained_agent,
        'reinforce_rewards': reinforce_rewards,
        'seed': random_seed  # Store the random seed for reproducibility
    }

# Save the results runs as joblib file
if not os.path.exists(output_path):
    os.makedirs(output_path)

from joblib import dump
with open(f"{output_path}/reinforce_results_runs_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib", 'wb') as f:
    dump(results_runs, f)



In [ ]:
# Define the environment
from utility_functions import  seed_all
import random
CONFIG_PATH = r'config/run_params.yaml'
MODEL_PATH = "oracles/oracle_3_GradientBoostingRegressor.joblib"
sys.path.append('../.')
config, env, logger, training_parameters, initial_state, feature_names = setup_environment_and_logger(CONFIG_PATH, MODEL_PATH)
# Correct state_dim to exclude the time-step part for action selection
state_dim = len(initial_state) + 1   # Include time-step part for the environment
action_dim = len(initial_state)     # Exclude time-step part for actions
output_path = r"results/PaperPlots_O3_REINFORCE_B_GBR"
results_runs = {}
for run in range(30):
    print(f"Run {run+1}/30")
    # Simulate trajectories using the forward and backward models
    random_seed = random.randint(0, 10000)    
    # with Temperature
    trained_agent,reinforce_rewards= train_rl_agent(
        env=env,
        state_dim=state_dim,
        action_dim=action_dim,
        algorithm='reinforce_baseline',
        training_episodes=500,
        max_steps=training_parameters["trajectory_length"],
        sac_params = {
            "alpha": 0.5,  # Temperature parameter for SAC
            "tau": 0.0005,  # Soft update parameter for SAC
            "gamma": 0.999,  # Discount factor for SAC
        },
        seed =  random_seed #33#34
    )
    # Store the trained agent and rewards
    results_runs[run] = {
        'trained_agent': trained_agent,
        'reinforce_rewards': reinforce_rewards,
        'seed': random_seed  # Store the random seed for reproducibility
    }

# Save the results runs as joblib file
if not os.path.exists(output_path):
    os.makedirs(output_path)

from joblib import dump
with open(f"{output_path}/reinforce_results_runs_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib", 'wb') as f:
    dump(results_runs, f)



In [ ]:
# Define the environment
from utility_functions import  seed_all
import random
CONFIG_PATH = r'config/run_params.yaml'
MODEL_PATH = "oracles/oracle_3_MLPRegressor.joblib"
sys.path.append('../.')
config, env, logger, training_parameters, initial_state, feature_names = setup_environment_and_logger(CONFIG_PATH, MODEL_PATH)
# Correct state_dim to exclude the time-step part for action selection
state_dim = len(initial_state) + 1   # Include time-step part for the environment
action_dim = len(initial_state)     # Exclude time-step part for actions
output_path = r"results/PaperPlots_O3_SAC_GBR"
results_runs = {}
for run in range(30):
    print(f"Run {run+1}/30")
    # Simulate trajectories using the forward and backward models
    random_seed = random.randint(0, 10000)    
    # with Temperature
    trained_agent,reinforce_rewards= train_rl_agent(
        env=env,
        state_dim=state_dim,
        action_dim=action_dim,
        algorithm='sac',
        training_episodes=500,
        max_steps=training_parameters["trajectory_length"],
        sac_params = {
            "alpha": 0.5,  # Temperature parameter for SAC
            "tau": 0.0005,  # Soft update parameter for SAC
            "gamma": 0.999,  # Discount factor for SAC
        },
        seed =  random_seed #33#34
    )
    # Store the trained agent and rewards
    results_runs[run] = {
        'trained_agent': trained_agent,
        'reinforce_rewards': reinforce_rewards,
        'seed': random_seed  # Store the random seed for reproducibility
    }

# Save the results runs as joblib file
if not os.path.exists(output_path):
    os.makedirs(output_path)

from joblib import dump
with open(f"{output_path}/sac_results_runs_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib", 'wb') as f:
    dump(results_runs, f)



In [ ]:
# Define the environment
CONFIG_PATH = r'config/run_params.yaml'
MODEL_PATH = "oracles/oracle_3_MLPRegressor.joblib"
sys.path.append('../.')
config, env, logger, training_parameters, initial_state, feature_names = setup_environment_and_logger(CONFIG_PATH, MODEL_PATH)
# Correct state_dim to exclude the time-step part for action selection
state_dim = len(initial_state) + 1   # Include time-step part for the environment
action_dim = len(initial_state)     # Exclude time-step part for actions
output_path = r"results/PaperPlots_O3_SMCMC_GBR"
results_runs = {}
for run in range(30):
    print(f"Run {run+1}/30")
    # Simulate trajectories using the forward and backward models
    random_seed = random.randint(0, 10000)
    seed_all(random_seed)  # Set random seed for reproducibility

    # Generate top MCMC results
    trajctories, rewards, actions, feature_names,non_sorted_rewards = generate_top_mcmc_results(
        env=env,
        num_trajectories=2000,  # Number of trajectories
        num_steps=training_parameters["trajectory_length"],  # Trajectory length
        proposal_std=10,  # Standard deviation for MCMC proposal
        feature_names=['Timestamp']+feature_names,  # Feature names from the environment
        temperature=10,  # Temperature for MCMC,
        # seed=random_seed  # Random seed for reproducibility
    )
    rewards = rewards.squeeze()
    trajctories = trajctories.squeeze()
    actions = actions.squeeze()
    last_states = np.array([traj[-1] for traj in trajctories])
    model_identifier = MODEL_PATH.split('.')[0][-8:]
    # Save MCMC results
    # Keep top 200
    trajctories = trajctories[:200]
    rewards = rewards[:200]
    actions = actions[:200]
    last_states = last_states[:200]

    results_dict = {
        'trajectories': trajctories,
        'rewards': rewards,
        'actions': actions,
        'last_states': last_states,
        'feature_names': feature_names,
        'non_sorted_rewards': non_sorted_rewards,
    }

    results_runs[run] = {
        'results_dict': results_dict,
        'seed': random_seed  # Store the random seed for reproducibility
    }

if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Save as joblib file
dump(results_runs, f"{output_path}/mcmc_results_runs_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib")

    # Print mean reward for this run
print(f"Mean reward: {np.mean(rewards)}")
# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
# Save as joblib file
from joblib import dump
dump(results_dict, f"{output_path}/mcmc_results_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib")





In [ ]:
results_runs[0]

In [ ]:
total_rewards = 0
for key,value in results_runs.items():
    print(f"Run {key}: Seed: {value['seed']}, Final Reward: {value['reinforce_rewards'][-1]}")
    total_rewards += value['reinforce_rewards'][-1]
print(f"Mean Final Reward: {total_rewards/len(results_runs)}")

In [ ]:
# Generate high-reward trajectories
high_reward_trajectories, rewards, all_actions, f_names = generate_high_reward_trajectories(
        env,
        trained_agent=trained_agent,
        num_trajectories=200,
        max_steps=training_parameters["trajectory_length"],
    )
model_identifier = MODEL_PATH.split('.')[0][-8:]

# Extract last states from each trajectory
last_states = np.array([traj[-1] for traj in high_reward_trajectories])

# Create a dict with all data
results_dict = {
    'trajectories': high_reward_trajectories,
    'rewards': rewards,
    'actions': all_actions,
    'last_states': last_states,
    'feature_names': f_names
}

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Save as joblib file
dump(results_dict, f"{output_path}/reinforce_results_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib")
# analyze_and_visualize_results(
#     agent_name='reinforce',
#     trained_agent=trained_agent,
#     training_rewards=reinforce_rewards,
#     env=env,
#     feature_names=feature_names,
#     training_parameters=training_parameters,
#     output_path=output_path,
#     model_identifier=model_identifier
# )

In [ ]:
# Define the environment
CONFIG_PATH = r'config/run_params.yaml'
MODEL_PATH = "oracles/oracle_3.joblib"
sys.path.append('../.')
config, env, logger, training_parameters, initial_state, feature_names = setup_environment_and_logger(CONFIG_PATH, MODEL_PATH)
# Correct state_dim to exclude the time-step part for action selection
state_dim = len(initial_state) + 1   # Include time-step part for the environment
action_dim = len(initial_state)     # Exclude time-step part for actions
output_path = r"results/PaperPlots_O1_REINFORCE_B"

# with Temperature
trained_agent,reinforce_rewards= train_rl_agent(
    env=env,
    state_dim=state_dim,
    action_dim=action_dim,
    algorithm='reinforce_baseline',
    training_episodes=150,
    max_steps=training_parameters["trajectory_length"],
    sac_params = {
        "alpha": 0.5,  # Temperature parameter for SAC
        "tau": 0.0005,  # Soft update parameter for SAC
        "gamma": 0.999,  # Discount factor for SAC
    },
    seed =  34 #33#34
)
 # Generate high-reward trajectories
high_reward_trajectories, rewards, all_actions, f_names = generate_high_reward_trajectories(
        env,
        trained_agent=trained_agent,
        num_trajectories=200,
        max_steps=training_parameters["trajectory_length"],
    )
model_identifier = MODEL_PATH.split('.')[0][-8:]

# Extract last states from each trajectory
last_states = np.array([traj[-1] for traj in high_reward_trajectories])

# Create a dict with all data
results_dict = {
    'trajectories': high_reward_trajectories,
    'rewards': rewards,
    'actions': all_actions,
    'last_states': last_states,
    'feature_names': f_names
}

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Save as joblib file
dump(results_dict, f"{output_path}/reinforce_baseline_results_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib")
# analyze_and_visualize_results(
#     agent_name='reinforce',
#     trained_agent=trained_agent,
#     training_rewards=reinforce_rewards,
#     env=env,
#     feature_names=feature_names,
#     training_parameters=training_parameters,
#     output_path=output_path,
#     model_identifier=model_identifier
# )

In [ ]:
# Define the environment
CONFIG_PATH = r'config/run_params.yaml'
MODEL_PATH = "oracles/oracle_3.joblib"
sys.path.append('../.')
config, env, logger, training_parameters, initial_state, feature_names = setup_environment_and_logger(CONFIG_PATH, MODEL_PATH)
# Correct state_dim to exclude the time-step part for action selection
state_dim = len(initial_state) + 1   # Include time-step part for the environment
action_dim = len(initial_state)     # Exclude time-step part for actions
output_path = r"results/PaperPlots_O3_SAC"

# with Temperature
trained_agent,reinforce_rewards= train_rl_agent(
    env=env,
    state_dim=state_dim,
    action_dim=action_dim,
    algorithm='sac',
    training_episodes=550,
    max_steps=training_parameters["trajectory_length"],
    sac_params = {
        "alpha": 0.5,  # Temperature parameter for SAC
        "tau": 0.0005,  # Soft update parameter for SAC
        "gamma": 0.999,  # Discount factor for SAC
    },
    seed =  34 #33#34
)
 # Generate high-reward trajectories
high_reward_trajectories, rewards, all_actions, f_names = generate_high_reward_trajectories(
        env,
        trained_agent=trained_agent,
        num_trajectories=200,
        max_steps=training_parameters["trajectory_length"],
    )
model_identifier = MODEL_PATH.split('.')[0][-8:]

# Extract last states from each trajectory
last_states = np.array([traj[-1] for traj in high_reward_trajectories])

# Create a dict with all data
results_dict = {
    'trajectories': high_reward_trajectories,
    'rewards': rewards,
    'actions': all_actions,
    'last_states': last_states,
    'feature_names': f_names
}

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Save as joblib file
dump(results_dict, f"{output_path}/sac_results_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib")
# analyze_and_visualize_results(
#     agent_name='reinforce',
#     trained_agent=trained_agent,
#     training_rewards=reinforce_rewards,
#     env=env,
#     feature_names=feature_names,
#     training_parameters=training_parameters,
#     output_path=output_path,
#     model_identifier=model_identifier
# )

# SMCMC

In [ ]:
# Define the environment
CONFIG_PATH = r'config/run_params.yaml'
MODEL_PATH = "oracles/oracle_2.joblib"
sys.path.append('../.')
config, env, logger, training_parameters, initial_state, feature_names = setup_environment_and_logger(CONFIG_PATH, MODEL_PATH)
# Correct state_dim to exclude the time-step part for action selection
state_dim = len(initial_state) + 1   # Include time-step part for the environment
action_dim = len(initial_state)     # Exclude time-step part for actions
output_path = r"results/PaperPlots_O2_SMCMC"
results_runs = {}
for run in range(30):
    print(f"Run {run+1}/30")
    # Simulate trajectories using the forward and backward models
    random_seed = random.randint(0, 10000)
    seed_all(random_seed)  # Set random seed for reproaducibility

    # Generate top MCMC results
    trajctories, rewards, actions, feature_names,non_sorted_rewards = generate_top_mcmc_results(
        env=env,
        num_trajectories=2000,  # Number of trajectories
        num_steps=training_parameters["trajectory_length"],  # Trajectory length
        proposal_std=10,  # Standard deviation for MCMC proposal
        feature_names=['Timestamp']+feature_names,  # Feature names from the environment
        temperature=10,  # Temperature for MCMC,
        # seed=random_seed  # Random seed for reproducibility
    )
    rewards = rewards.squeeze()
    trajctories = trajctories.squeeze()
    actions = actions.squeeze()
    last_states = np.array([traj[-1] for traj in trajctories])
    model_identifier = MODEL_PATH.split('.')[0][-8:]
    # Save MCMC results
    # Keep top 200
    trajctories = trajctories[:200]
    rewards = rewards[:200]
    actions = actions[:200]
    last_states = last_states[:200]

    results_dict = {
        'trajectories': trajctories,
        'rewards': rewards,
        'actions': actions,
        'last_states': last_states,
        'feature_names': feature_names,
        'non_sorted_rewards': non_sorted_rewards,
    }

    results_runs[run] = {
        'results_dict': results_dict,
        'seed': random_seed  # Store the random seed for reproducibility
    }

if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Save as joblib file
dump(results_runs, f"{output_path}/mcmc_results_runs_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib")

    # Print mean reward for this run
print(f"Mean reward: {np.mean(rewards)}")
# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
# Save as joblib file
from joblib import dump
dump(results_dict, f"{output_path}/mcmc_results_{MODEL_PATH.split('/')[-1].split('.')[0]}.joblib")





In [ ]:
rewards[:,-1]

In [ ]:
results_dict['rewards']

In [ ]:
trajctories.shape, rewards.shape, actions.shape

In [ ]:
pd.Series(last_step_rewards, name='Final Rewards').to_csv(f"{output_path}/SMCMC_Final_Rewards.csv", index=False)

In [ ]:
pd.Series(last_step_reinforce_rewards, name='Final Rewards').to_csv(f"Reinforce_Final_Rewards.csv", index=False)

In [ ]:
cosine_distances.max()

In [ ]:
from scipy.spatial.distance import pdist, squareform

def calculate_euclidean_diversity(final_states: np.ndarray, rewards):
    distances = pdist(final_states, metric='euclidean')
    avg_div = distances.mean()
    avg_div_normalized = (avg_div / (max(rewards)-min(rewards))) * max(rewards)
    distances = squareform(distances)  # Convert to square form for better visualization
    # Convert to square form for better visualization
    return avg_div, avg_div_normalized,distances

In [ ]:
(avg_diversity, 
avg_div_normalized,
distances)= calculate_euclidean_diversity(top10_mcmc_df[feature_names].values,top10_mcmc_df['FinalReward'])

# Create plot
plt.figure(figsize=(8, 8))
sns.heatmap(distances, 
            cmap='viridis', 
            fmt=".2f",
            vmin=0,  # Set minimum value to 0
            vmax=500,  # Set maximum value to 500
            cbar_kws={'label': 'Euclidean Distance'})

plt.title(f"Euclidean Distance Matrix for Top 10 Trajectories\nDiversity Score: {avg_diversity:.4f}, Normalized: {avg_div_normalized[0]:.4f}")
plt.xlabel("Trajectory Index")
plt.ylabel("Trajectory Index")
plt.tight_layout()
# Save as high quality pdf
plt.savefig(f"{output_path}/smcmc_diversity_heatmap_{MODEL_PATH.split('/')[-1].split('.')[0]}.pdf", dpi=300)
plt.show()

# Doing it properly

In [ ]:
import os
import sys
import yaml
import joblib
import torch
import random
import numpy as np
from joblib import dump
from typing import Dict, Any, Callable

from utility_functions import seed_all


import sys
import yaml
import joblib
import torch
import random
import numpy as np
from joblib import dump
from typing import Dict, Any, Callable
from utility_functions import seed_all


def run_all_experiments(
    experiments: Dict[str, Dict[str, Any]],
    base_output_dir: str,
    simulate_fn: Callable = None,
) -> None:
    """
    Unified experiment runner that infers which algorithm to run based on run name.
    Supports RL, SMCMC, and GFlowNet runs with per-run configurations.

    Args:
        experiments: Dictionary mapping run_name -> parameters. Expected keys:
            - oracle_path (for RL/SMCMC)
            - config_path (for RL/SMCMC)
            - num_runs, training_episodes, etc.
            - For SMCMC: n_trajectories, proposal_std, temperature.
            - For GFlow: fwd_model_path, bwd_model_path, run_params_path, gflow_runs (list of dicts).
        base_output_dir: Root folder where results will be stored.
        simulate_fn: Function to simulate GFlow trajectories (required for gflow runs).
    """
    sys.path.append('../.')

    for run_name, params in experiments.items():
        print(f"\n=== Running experiment: {run_name} ===")
        output_path = os.path.join(base_output_dir, run_name)
        os.makedirs(output_path, exist_ok=True)

        run_name_lower = run_name.lower()
        # --- Determine algorithm type ---
        if "gflow" in run_name_lower:
            algo_name = "gflow"
        elif "smcmc" in run_name_lower:
            algo_name = "smcmc"
        elif "reinforce_baseline" in run_name_lower:
            algo_name = "reinforce_baseline"
        elif "reinforce" in run_name_lower:
            algo_name = "reinforce"
        elif "sac" in run_name_lower:
            algo_name = "sac"
        else:
            raise ValueError(f"Cannot detect algorithm type from run name: {run_name}")

        # === RL/SMCMC Experiments ===
        if algo_name in {"reinforce", "reinforce_baseline", "sac", "smcmc"}:
            config_path = params.get("config_path")
            if not config_path:
                raise ValueError(f"No config_path provided for run: {run_name}")

            config, env, logger, training_parameters, initial_state, feature_names = (
                setup_environment_and_logger(config_path, params["oracle_path"])
            )

            state_dim = len(initial_state) + 1
            action_dim = len(initial_state)
            results_runs = {}

            for run in range(params.get("num_runs", 30)):
                print(f"Run {run+1}/{params.get('num_runs', 30)}")
                random_seed = random.randint(0, 10_000)
                seed_all(random_seed)

                if algo_name in {"reinforce", "reinforce_baseline", "sac"}:
                    trained_agent, rewards = train_rl_agent(
                        env=env,
                        state_dim=state_dim,
                        action_dim=action_dim,
                        algorithm=algo_name,
                        training_episodes=params.get("training_episodes", 500),
                        max_steps=training_parameters["trajectory_length"],
                        sac_params={
                            "alpha": 0.5,
                            "tau": 0.0005,
                            "gamma": 0.999,
                        },
                        seed=random_seed,
                    )
                    results_runs[run] = {
                        "trained_agent": trained_agent,
                        "rewards": rewards,
                        "seed": random_seed,
                    }

                elif algo_name == "smcmc":
                    # === EXACT MATCH to your original saving style ===
                    trajectories, rewards, actions, feature_names_out, non_sorted_rewards = (
                        generate_top_mcmc_results(
                            env=env,
                            num_trajectories=params.get("n_trajectories", 2000),
                            num_steps=training_parameters["trajectory_length"],
                            proposal_std=params.get("proposal_std", 10),
                            feature_names=['Timestamp'] + feature_names,
                            temperature=params.get("temperature", 10),
                        )
                    )

                    rewards = rewards.squeeze()
                    trajectories = trajectories.squeeze()
                    actions = actions.squeeze()
                    last_states = np.array([traj[-1] for traj in trajectories])

                    # Keep top 200 trajectories
                    trajectories = trajectories[:200]
                    rewards = rewards[:200]
                    actions = actions[:200]
                    last_states = last_states[:200]

                    results_dict = {
                        "trajectories": trajectories,
                        "rewards": rewards,
                        "actions": actions,
                        "last_states": last_states,
                        "feature_names": feature_names_out,
                        "non_sorted_rewards": non_sorted_rewards,
                    }

                    results_runs[run] = {
                        "results_dict": results_dict,
                        "seed": random_seed,
                    }

            dump(results_runs, os.path.join(output_path, f"{algo_name}_results.joblib"))
            print(f"Saved {algo_name} results to {output_path}")

        # === GFlowNet Experiments ===
        elif algo_name == "gflow":
            if not simulate_fn:
                raise RuntimeError("simulate_fn must be provided for GFlow runs")

            results_runs = {}
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

            # Support per-run config path for GFlow
            run_params_path = params.get("run_params_path")
            if not run_params_path:
                raise ValueError(f"No run_params_path provided for GFlow run: {run_name}")
            config = yaml.safe_load(open(run_params_path, "r"))

            for i, run_cfg in enumerate(params.get("gflow_runs", [])):
                seed = run_cfg["seed"]
                seed_all(seed)

                print(f"Simulating GFlow run {i+1}/{len(params['gflow_runs'])} with seed {seed}...")
                forward_model = load_entire_model(params["fwd_model_path"], device)
                backward_model = load_entire_model(params["bwd_model_path"], device)
                logger, _ = setup_logger(f"gflow_run_{i}")
                initial_state, feature_names = load_initial_state(config)

                trajectories, rewards, all_actions, _ = simulate_fn(
                    env_class=GeneralEnvironment,
                    forward_model=forward_model,
                    backward_model=backward_model,
                    initial_state=initial_state,
                    config=config,
                    trajectory_length=run_cfg.get("trajectory_length", 12),
                    n_trajectories=run_cfg.get("n_trajectories", 200),
                    device=device,
                    logger=logger,
                    model_path=config["oracle"]["model_path"],
                    distribution="mixture_beta",
                    extra_parameters=params.get("extra_parameters", {}),
                )

                results_runs[i] = {
                    "trajectories": trajectories,
                    "rewards": np.array(rewards),
                    "all_actions": all_actions,
                    "seed": seed,
                }

            joblib.dump(results_runs, os.path.join(output_path, "gflow_results_runs.joblib"))
            print(f"GFlow results saved to {output_path}")


In [ ]:
EXPERIMENTS = {
    "reinforce_1_mlp": {
        "oracle_path": "oracles/oracle_1_MLPRegressor.joblib",
        "config_path": "config/_run_params_1.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "reinforce_2_mlp": {
        "oracle_path": "oracles/oracle_2_MLPRegressor.joblib",
        "config_path": "config/_run_params_2.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "reinforce_3_mlp": {
        "oracle_path": "oracles/oracle_3_MLPRegressor.joblib",
        "config_path": "config/_run_params_3.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "reinforce_baseline_1_mlp": {
        "oracle_path": "oracles/oracle_1_MLPRegressor.joblib",
        "config_path": "config/_run_params_1.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "reinforce_baseline_2_mlp": {
        "oracle_path": "oracles/oracle_2_MLPRegressor.joblib",
        "config_path": "config/_run_params_2.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "reinforce_baseline_3_mlp": {
        "oracle_path": "oracles/oracle_3_MLPRegressor.joblib",
        "config_path":"config/_run_params_3.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "sac_1_mlp": {
        "oracle_path": "oracles/oracle_1_MLPRegressor.joblib",
        "config_path": "config/_run_params_1.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "sac_2_mlp": {
        "oracle_path": "oracles/oracle_2_MLPRegressor.joblib",
        "config_path": "config/_run_params_2.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "sac_3_mlp": {
        "oracle_path": "oracles/oracle_3_MLPRegressor.joblib",
        "config_path": "config/_run_params_3.yaml",
        "num_runs": 30,
        "training_episodes": 500,
    },
    "smcmc_1_mlp": {
        "oracle_path": "oracles/oracle_1_MLPRegressor.joblib",
        "config_path": "config/_run_params_1.yaml",
        "num_runs": 30,
        "n_trajectories": 2000,
        "proposal_std": 10,
    },
    "smcmc_2_mlp": {
        "oracle_path": "oracles/oracle_2_MLPRegressor.joblib",
        "config_path": "config/_run_params_2.yaml",
        "num_runs": 30,
        "n_trajectories": 2000,
        "proposal_std": 10,
    },
    "smcmc_3_mlp": {
            "oracle_path": "oracles/oracle_3_MLPRegressor.joblib",
            "config_path": "config/_run_params_3.yaml",
            "num_runs": 30,
            "n_trajectories": 2000,
            "proposal_std": 10,
        },

}


# run_all_experiments(
#     experiments=EXPERIMENTS,
#     base_output_dir='results/All_Experiment_Results',
#     simulate_fn=None,
# )
   

In [ ]:
EXPERIMENTS["gflow_1_mlp"] = {
    "gflow_results_path": "mlruns/2/240c95ed1dfa41de9637a8d9a876e50d/artifacts/forward_model_iteration_500/results_run.joblib",
    "run_params_path": "config/_run_params_1.yaml",
    "oracle_path": "oracles/oracle_1_MLPRegressor.joblib",
    "trajectory_length": 12,
    "n_trajectories": 200,
    "extra_parameters": {
        'mixture_components': 15,
        'num_variables': 6,
    }
}
EXPERIMENTS["gflow_2_mlp"] = {
    "gflow_results_path": "mlruns/3/385afeb31ff1439b834d885958294961/artifacts/forward_model_iteration_500/results_run.joblib",
    "run_params_path": "config/_run_params_2.yaml",
    "oracle_path": "oracles/oracle_2_MLPRegressor.joblib",
    "trajectory_length": 12,
    "n_trajectories": 200,
    "extra_parameters": {
        'mixture_components': 15,
        'num_variables': 6,
    }
}

EXPERIMENTS['gflow_3_mlp'] = {
    "gflow_results_path": "mlruns/4/785cadee6b0141ad9de43ea9b1cf365c/artifacts/forward_model_iteration_500/results_run.joblib",
    "run_params_path": "config/_run_params_3.yaml",
    "oracle_path": "oracles/oracle_3_MLPRegressor.joblib",
    "trajectory_length": 12,
    "n_trajectories": 200,
    "extra_parameters": {
        'mixture_components': 15,
        'num_variables': 6,
    }
}

In [ ]:
EXPERIMENTS

In [ ]:
import copy
import re
from typing import Dict, Any, Iterable, Tuple

# Map short tag -> sklearn model suffix used in filenames
MODEL_SUFFIX_MAP = {
    "mlp": "MLPRegressor",
    "gbr": "GradientBoostingRegressor",
    "rfr": "RandomForestRegressor",
    "elastic":"ElasticNet",
    "ens":"EnsembleRegressor",
}


def _replace_oracle_suffix(path: str, new_suffix: str) -> str:
    """Replace the trailing model suffix in an oracle path.

    It expects filenames like: .../oracle_3_<ModelSuffix>.joblib
    Supported suffixes: MLPRegressor | GradientBoostingRegressor | RandomForestRegressor

    Args:
        path: Original oracle .joblib path.
        new_suffix: New model suffix to inject, e.g., "GradientBoostingRegressor".

    Returns:
        Modified path with the new suffix.

    Raises:
        ValueError: If the path does not match the expected pattern.
    """
    new_path = re.sub(
        r"_(MLPRegressor|GradientBoostingRegressor|RandomForestRegressor)\.joblib$",
        f"_{new_suffix}.joblib",
        path,
    )
    if new_path == path:
        raise ValueError(
            f"oracle_path does not end with a supported suffix: {path}"
        )
    return new_path


def augment_experiments_with_variants(
    experiments: Dict[str, Dict[str, Any]],
    variants: Iterable[str] = ("gbr", "rfr"),
) -> Dict[str, Dict[str, Any]]:
    """Create GBR/RFR copies from existing *_mlp runs by updating only oracle_path.

    For each key that ends with `_mlp`, this will create parallel entries with
    `_gbr` and `_rfr` (or whatever `variants` you pass). If an entry already
    exists, it is left untouched.

    Args:
        experiments: Base experiments dict (will NOT be mutated).
        variants: Iterable of short tags to create (e.g., ("gbr", "rfr")).

    Returns:
        A NEW dict containing the original experiments plus the added variants.
    """
    out: Dict[str, Dict[str, Any]] = copy.deepcopy(experiments)

    for run_name, cfg in experiments.items():
        # Only clone runs that are *oracle-based* and keyed as *_mlp
        if not run_name.endswith("_mlp"):
            continue
        if "oracle_path" not in cfg:
            continue  # skip non-oracle entries like gflow, etc.

        base_oracle_path = cfg["oracle_path"]

        for v in variants:
            if v not in MODEL_SUFFIX_MAP:
                raise KeyError(f"Unknown variant '{v}'. "
                               f"Known: {list(MODEL_SUFFIX_MAP.keys())}")

            new_key = run_name[:-4] + f"_{v}"  # replace trailing _mlp with _gbr/_rfr
            if new_key in out:
                # already present; do not overwrite
                continue

            new_cfg = copy.deepcopy(cfg)
            new_cfg["oracle_path"] = _replace_oracle_suffix(
                base_oracle_path, MODEL_SUFFIX_MAP[v]
            )
            out[new_key] = new_cfg

    return out

EXPERIMENTS = augment_experiments_with_variants(EXPERIMENTS, variants=("elastic", "ens","gbr", "rfr"))


In [ ]:
# Filter out keys containing 'mlp' or 'gflow' also fil
filtered_experiments = {
    key: value for key, value in EXPERIMENTS.items() 
    if 'mlp' not in key.lower() and 'gflow' not in key.lower()
}
print(f"Remaining experiments: {list(filtered_experiments.keys())}")

In [ ]:
# Filter to only keep keys containing 'ens' or 'elastic'
filtered_experiments = {
    key: value for key, value in EXPERIMENTS.items() 
    if 'ens' in key.lower() or 'elastic' in key.lower()
}
print(f"Remaining experiments: {list(filtered_experiments.keys())}")

In [ ]:
# Extract all SMCMC experiments from EXPERIMENTS
smcmc_experiments = {
    key: value for key, value in EXPERIMENTS.items() 
    if 'smcmc' in key.lower()
}
print(f"SMCMC experiments: {list(smcmc_experiments.keys())}")

In [ ]:
# Add SMCMC experiments to filtered_experiments
filtered_experiments.update(smcmc_experiments)
print(f"Updated filtered_experiments with SMCMC. Total experiments: {len(filtered_experiments)}")
print(f"Keys: {list(filtered_experiments.keys())}")

In [ ]:
import os
import sys
import joblib
import numpy as np
import torch
import yaml
import random
from joblib import dump

from utility_functions import seed_all


def simulate_all_trajectories(
    experiments: dict,
    base_output_dir: str,
    num_trajectories: int = 200,
) -> None:
    """
    After training, load results and generate/simulate trajectories for each experiment.

    Args:
        experiments: Same dictionary passed to run_all_experiments().
            For GFlow, dict should include:
              - fwd_model_path
              - bwd_model_path
              - run_params_path
              - num_runs
              - trajectory_length (optional)
              - n_trajectories (optional)
              - extra_parameters (optional)
        base_output_dir: Root folder where training results are stored.
        num_trajectories: Number of trajectories to simulate per RL agent (ignored for GFlow if overridden).

    Returns:
        None. Saves a `trajectories_results.joblib` file per run.
    """
    sys.path.append('../.')

    for run_name, params in experiments.items():
        print(f"\n=== Simulating trajectories for: {run_name} ===")
        run_output_dir = os.path.join(base_output_dir, run_name)
        os.makedirs(run_output_dir, exist_ok=True)

        run_name_lower = run_name.lower()

        if "gflow" in run_name_lower:
            algo = "gflow"
        elif "reinforce" in run_name_lower or "sac" in run_name_lower:
            algo = "rl"
        elif "smcmc" in run_name_lower:
            algo = "smcmc"
        else:
            print(f"Skipping {run_name} (unsupported for simulation)")
            continue

        print(f"Oracle Used: {params['oracle_path']}")

        if algo == "rl":
            results_file = os.path.join(run_output_dir, f"{run_name_lower.split('_')[0]}_results.joblib")
            print(f"Loading RL results from {results_file}...")
            if not os.path.exists(results_file):
                print(f"Skipping {run_name}, results file not found.")
                continue

            results_runs = joblib.load(results_file)
            config, env, logger, training_parameters, initial_state, feature_names = (
                setup_environment_and_logger(params["config_path"], params["oracle_path"])
            )

            traj_results = {}
            for run_idx, run_data in results_runs.items():
                trained_agent = run_data["trained_agent"]
                print(f"  Generating RL trajectories for run {run_idx}...")
                trajectories, rewards, all_actions, f_names = generate_high_reward_trajectories(
                    env,
                    trained_agent=trained_agent,
                    num_trajectories=num_trajectories,
                    max_steps=training_parameters["trajectory_length"],
                )
                last_states = np.array([traj[-1] for traj in trajectories])
                traj_results[run_idx] = {
                    "trajectories": trajectories,
                    "rewards": rewards,
                    "actions": all_actions,
                    "last_states": last_states,
                    "feature_names": f_names,
                    "seed": run_data.get("seed"),
                }

            dump(traj_results, os.path.join(run_output_dir, "trajectories_results.joblib"))
            print(f"Saved RL trajectory simulations for {run_name}")

        elif algo == "smcmc":
            results_file = os.path.join(run_output_dir, "smcmc_results.joblib")
            if not os.path.exists(results_file):
                print(f"Skipping {run_name}, SMCMC results file not found.")
                continue

            results_runs = joblib.load(results_file)
            summarized_results = {}
            for run_idx, run_data in results_runs.items():
                results_dict = run_data["results_dict"]
                summarized_results[run_idx] = {
                    "trajectories": results_dict["trajectories"],
                    "rewards": results_dict["rewards"],
                    "actions": results_dict["actions"],
                    "last_states": results_dict["last_states"],
                    "feature_names": results_dict["feature_names"],
                    "seed": run_data.get("seed"),
                }

            dump(summarized_results, os.path.join(run_output_dir, "trajectories_results.joblib"))
            print(f"Saved SMCMC summarized trajectories for {run_name}")

        elif algo == "gflow":
            from utility_functions import load_initial_state, setup_logger, simulate_trajectories

            # Load the joblib file containing all trained forward/backward models
            models_dict = joblib.load(params["gflow_results_path"])
            config = yaml.safe_load(open(params["run_params_path"], "r"))
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            results_runs = {}

            for run_idx, run_data in models_dict.items():
                seed = run_data["seed"]
                seed_all(seed)
                print(f"  Generating GFlow trajectories for run {run_idx+1}/{len(models_dict)} (seed={seed})...")

                forward_model = run_data["trained_agent"]
                backward_model = run_data["trained_backward_agent"]

                logger, _ = setup_logger(f"gflow_run_{run_idx}")
                initial_state, feature_names = load_initial_state(config)
                extra_params = params.get("extra_parameters", {})
                print(f"Oracle Used: {params['oracle_path']}")
                trajectories, rewards, all_actions, _ = simulate_trajectories(
                    env_class=GeneralEnvironment,
                    forward_model=forward_model,
                    backward_model=backward_model,
                    initial_state=initial_state,
                    config=config,
                    trajectory_length=params.get("trajectory_length", 12),
                    n_trajectories=params.get("n_trajectories", 200),
                    device=device,
                    logger=logger,
                    model_path=params["oracle_path"],
                    distribution="mixture_beta",
                    extra_parameters=extra_params,
                )

                results_runs[run_idx] = {
                    "trajectories": trajectories,
                    "rewards": np.array(rewards),
                    "all_actions": all_actions,
                    "seed": seed,
                }

            dump(results_runs, os.path.join(run_output_dir, f"{run_name}_trajectories_results.joblib"))
            print(f"Saved GFlow trajectories for {run_name}")



simulate_all_trajectories(
    experiments=filtered_experiments,
    base_output_dir='results/All_Experiment_Results',
    num_trajectories=200,
)


In [ ]:
def simulate_all_trajectories(
    experiments: dict,
    base_output_dir: str,
    num_trajectories: int = 200,
) -> None:
    """
    Unified simulation runner.
    It loads the experiment configuration and then, based on the run name,
    simulates trajectories using RL, SMCMC or GFlowNet approaches.

    For SMCMC (which has no external model to load), the trajectories are generated
    from scratch by calling generate_top_mcmc_results.
    
    Args:
        experiments: Dictionary mapping run name to parameters.
        base_output_dir: Root folder where simulation results are stored.
        num_trajectories: Number of trajectories to simulate per run (for RL; SMCMC uses its own parameters).
    
    Returns:
        None. Results are saved as joblib files.
    """
    sys.path.append('../.')
    for run_name, params in experiments.items():
        print(f"\n=== Simulating trajectories for: {run_name} ===")
        run_output_dir = os.path.join(base_output_dir, run_name)
        os.makedirs(run_output_dir, exist_ok=True)

        run_name_lower = run_name.lower()
        if "gflow" in run_name_lower:
            algo = "gflow"
        elif "reinforce" in run_name_lower or "sac" in run_name_lower:
            algo = "rl"
        elif "smcmc" in run_name_lower:
            algo = "smcmc"
        else:
            print(f"Skipping {run_name} (unsupported for simulation)")
            continue

        print(f"Oracle Used: {params['oracle_path']}")

        if algo == "rl":
            results_file = os.path.join(run_output_dir, f"{run_name_lower.split('_')[0]}_results.joblib")
            if not os.path.exists(results_file):
                print(f"Skipping {run_name}, results file not found.")
                continue
            results_runs = joblib.load(results_file)
            config, env, logger, training_parameters, initial_state, feature_names = (
                setup_environment_and_logger(params["config_path"], params["oracle_path"])
            )
            traj_results = {}
            for run_idx, run_data in results_runs.items():
                trained_agent = run_data["trained_agent"]
                print(f"  Generating RL trajectories for run {run_idx}...")
                # Replace generate_high_reward_trajectories with your RL function if needed.
                trajectories, rewards, all_actions, f_names = generate_high_reward_trajectories(
                    env,
                    trained_agent=trained_agent,
                    num_trajectories=num_trajectories,
                    max_steps=training_parameters["trajectory_length"],
                )
                last_states = np.array([traj[-1] for traj in trajectories])
                traj_results[run_idx] = {
                    "trajectories": trajectories,
                    "rewards": rewards,
                    "actions": all_actions,
                    "last_states": last_states,
                    "feature_names": f_names,
                    "seed": run_data.get("seed"),
                }
            dump(traj_results, os.path.join(run_output_dir, "trajectories_results.joblib"))
            print(f"Saved RL trajectory simulations for {run_name}")

        elif algo == "smcmc":
            config, env, logger, training_parameters, initial_state, feature_names = (
                setup_environment_and_logger(params["config_path"], params["oracle_path"])
            )
            # For SMCMC, generate trajectories from scratch using your SMCMC function.
            results_runs = {}
            for run in range(params.get("num_runs", 30)):
                print(f"  SMCMC Run {run+1}/{params.get('num_runs', 30)}")
                random_seed = random.randint(0, 10000)
                seed_all(random_seed)
                trajs, rewards, actions, fname_out, non_sorted_rewards = generate_top_mcmc_results(
                    env=env,
                    num_trajectories=params.get("n_trajectories", 2000),
                    num_steps=training_parameters["trajectory_length"],
                    proposal_std=params.get("proposal_std", 10),
                    feature_names=['Timestamp'] + feature_names,
                    temperature=params.get("temperature", 10),
                )
                # Squeeze, if needed
                rewards = rewards.squeeze()
                trajs = trajs.squeeze()
                actions = actions.squeeze()
                last_states = np.array([traj[-1] for traj in trajs])
                # Keep top 200 trajectories
                trajs = trajs[:200]
                rewards = rewards[:200]
                actions = actions[:200]
                last_states = last_states[:200]
                results_dict = {
                    "trajectories": trajs,
                    "rewards": rewards,
                    "actions": actions,
                    "last_states": last_states,
                    "feature_names": fname_out,
                    "non_sorted_rewards": non_sorted_rewards,
                }
                results_runs[run] = {
                    "results_dict": results_dict,
                    "seed": random_seed,
                }
            dump(results_runs, os.path.join(run_output_dir, "trajectories_results.joblib"))
            print(f"Saved SMCMC trajectory simulations for {run_name}")

        elif algo == "gflow":
            # (GFlow branch remains unchanged; include your GFlow simulation code here.)
            from utility_functions import load_initial_state, setup_logger, simulate_trajectories
            models_dict = joblib.load(params["gflow_results_path"])
            config = yaml.safe_load(open(params["run_params_path"], "r"))
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            results_runs = {}
            for run_idx, run_data in models_dict.items():
                seed = run_data["seed"]
                seed_all(seed)
                print(f"  Generating GFlow trajectories for run {run_idx+1}/{len(models_dict)} (seed={seed})...")
                forward_model = run_data["trained_agent"]
                backward_model = run_data["trained_backward_agent"]
                logger, _ = setup_logger(f"gflow_run_{run_idx}")
                initial_state, feature_names = load_initial_state(config)
                extra_params = params.get("extra_parameters", {})
                trajectories, rewards, all_actions, _ = simulate_trajectories(
                    env_class=GeneralEnvironment,
                    forward_model=forward_model,
                    backward_model=backward_model,
                    initial_state=initial_state,
                    config=config,
                    trajectory_length=params.get("trajectory_length", 12),
                    n_trajectories=params.get("n_trajectories", 200),
                    device=device,
                    logger=logger,
                    model_path=params["oracle_path"],
                    distribution="mixture_beta",
                    extra_parameters=extra_params,
                )
                results_runs[run_idx] = {
                    "trajectories": trajectories,
                    "rewards": np.array(rewards),
                    "all_actions": all_actions,
                    "seed": seed,
                }
            dump(results_runs, os.path.join(run_output_dir, f"{run_name}_trajectories_results.joblib"))
            print(f"Saved GFlow trajectories for {run_name}")
            
simulate_all_trajectories(
    experiments=filtered_experiments,
    base_output_dir='results/All_Experiment_Results',
    num_trajectories=200,
)

In [ ]:
# Generate high-reward trajectories
high_reward_trajectories, rewards, all_actions, f_names = generate_high_reward_trajectories(
        env,
        trained_agent=trained_agent,
        num_trajectories=200,
        max_steps=12,
    )
model_identifier = MODEL_PATH.split('.')[0][-8:]

# Extract last states from each trajectory
last_states = np.array([traj[-1] for traj in high_reward_trajectories])

# Create a dict with all data
results_dict = {
    'trajectories': high_reward_trajectories,
    'rewards': rewards,
    'actions': all_actions,
    'last_states': last_states,
    'feature_names': f_names
}

# Create output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)